# Imports

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv1D, MaxPooling1D

# Train

In [2]:
train_set = pd.read_csv('train/train_encoded.csv')

In [3]:
train_set.drop('id', axis=1, inplace=True)

In [4]:
X, y = train_set.iloc[:,:-1],train_set.iloc[:,-1]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

# Model

In [5]:
batch_size = 1
num_classes = 2
epochs = 10

In [6]:
X_train

,len_keyword,len_location,len_text,len_text_original,diff_len_text,keyword_sum,keyword_min,keyword_max,keyword_mean,keyword_median,...,text_sum_tf-idf,text_min_tf-idf,text_max_tf-idf,text_mean_tf-idf,text_median_tf-idf,text_sum_w2v,text_min_w2v,text_max_w2v,text_mean_w2v,text_median_w2v
5342,11.0,17.0,64,88,24,1.000000,1.000000,1.000000,1.000000,1.000000,...,3.075644,0.126763,0.377990,0.307564,0.329847,3.893729,-0.055620,0.431871,0.086527,0.063590
6337,15.0,24.0,95,117,22,1.414214,0.707107,0.707107,0.707107,0.707107,...,3.361030,0.100555,0.367202,0.280086,0.285886,4.414135,-0.053262,0.268294,0.080257,0.071987
915,6.0,28.0,105,129,24,1.000000,1.000000,1.000000,1.000000,1.000000,...,3.782429,0.089744,0.357183,0.222496,0.233757,10.905544,-0.040835,0.497005,0.119841,0.108092
5690,5.0,28.0,104,107,3,1.000000,1.000000,1.000000,1.000000,1.000000,...,5.714941,0.086989,0.457572,0.285747,0.262499,38.922700,-0.041069,1.000000,0.286196,0.238599
3029,9.0,0.0,98,121,23,1.000000,1.000000,1.000000,1.000000,1.000000,...,3.289223,0.085256,0.346145,0.274102,0.322001,1.882967,-0.016486,0.282174,0.089665,0.091887
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1225,10.0,13.0,114,117,3,1.413688,0.687569,0.726119,0.706844,0.706844,...,4.449931,0.083032,0.337116,0.202270,0.189729,16.265894,-0.063523,0.482684,0.106313,0.089853
3215,12.0,8.0,66,88,22,1.402200,0.609129,0.793071,0.701100,0.701100,...,3.045940,0.108639,0.432385,0.276904,0.268736,4.455212,-0.002509,0.363886,0.123756,0.101253
5789,4.0,18.0,128,128,0,1.000000,1.000000,1.000000,1.000000,1.000000,...,4.911973,0.089455,0.351161,0.213564,0.218253,48.070689,-0.068548,1.000000,0.228908,0.189672
349,4.0,0.0,44,89,45,1.000000,1.000000,1.000000,1.000000,1.000000,...,2.837199,0.141578,0.433233,0.315244,0.360362,2.613822,0.004426,0.376678,0.124468,0.121230


In [7]:
X_train = np.expand_dims(X_train, axis=2)
X_test = np.expand_dims(X_test, axis=2)

In [8]:
X_train.shape

(5329, 25, 1)

In [9]:
input_shape = (X_train.shape[1], 1)
input_shape

(25, 1)

## Relu

In [10]:
model = Sequential()
model.add(Conv1D(128, kernel_size=2,
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv1D(64, 1, activation='relu'))
model.add(MaxPooling1D(pool_size=3))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='relu'))

model.compile(loss=keras.losses.binary_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

In [11]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d (Conv1D)              (None, 24, 128)           384       
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 24, 64)            8256      
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 8, 64)             0         
_________________________________________________________________
dropout (Dropout)            (None, 8, 64)             0         
_________________________________________________________________
flatten (Flatten)            (None, 512)               0         
_________________________________________________________________
dense (Dense)                (None, 128)               65664     
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0

In [12]:
model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=5,
          verbose=1,
          validation_data=(X_test, y_test))

Epoch 1/5
5329/5329 [==============================] - 15s 3ms/step - loss: 6.8646 - accuracy: 0.5035 - val_loss: 4.4601 - val_accuracy: 0.5381
Epoch 2/5
5329/5329 [==============================] - 15s 3ms/step - loss: 6.8202 - accuracy: 0.5112 - val_loss: 4.6010 - val_accuracy: 0.5679
Epoch 3/5
5329/5329 [==============================] - 15s 3ms/step - loss: 6.6942 - accuracy: 0.5151 - val_loss: 4.7952 - val_accuracy: 0.5880
Epoch 4/5
5329/5329 [==============================] - 15s 3ms/step - loss: 6.7665 - accuracy: 0.5115 - val_loss: 5.1324 - val_accuracy: 0.5827
Epoch 5/5
5329/5329 [==============================] - 15s 3ms/step - loss: 6.7819 - accuracy: 0.5061 - val_loss: 5.5604 - val_accuracy: 0.5771


In [29]:
model = Sequential()
model.add(Conv1D(128, kernel_size=3,
                 activation='relu',
                 input_shape=input_shape))
model.add(MaxPooling1D(pool_size=4))
model.add(Dropout(0.25))
model.add(Dense(128))
model.add(Conv1D(filters=32, kernel_size=1, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(64, 1))
model.add(MaxPooling1D(pool_size=1))
model.add(Flatten())
model.add(Dense(num_classes, activation='relu'))

model.compile(loss=keras.losses.binary_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

In [30]:
model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_21 (Conv1D)           (None, 23, 128)           512       
_________________________________________________________________
max_pooling1d_18 (MaxPooling (None, 5, 128)            0         
_________________________________________________________________
dropout_10 (Dropout)         (None, 5, 128)            0         
_________________________________________________________________
dense_14 (Dense)             (None, 5, 128)            16512     
_________________________________________________________________
conv1d_22 (Conv1D)           (None, 5, 32)             4128      
_________________________________________________________________
max_pooling1d_19 (MaxPooling (None, 2, 32)             0         
_________________________________________________________________
conv1d_23 (Conv1D)           (None, 2, 64)            

In [31]:
model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=5,
          verbose=1,
          validation_data=(X_test, y_test))

Epoch 1/5
5329/5329 [==============================] - 13s 3ms/step - loss: 6.2114 - accuracy: 0.5267 - val_loss: 4.4661 - val_accuracy: 0.5604
Epoch 2/5
5329/5329 [==============================] - 13s 2ms/step - loss: 6.1241 - accuracy: 0.5500 - val_loss: 4.3443 - val_accuracy: 0.5604
Epoch 3/5
5329/5329 [==============================] - 13s 2ms/step - loss: 6.1773 - accuracy: 0.5395 - val_loss: 4.3340 - val_accuracy: 0.5604
Epoch 4/5
5329/5329 [==============================] - 13s 2ms/step - loss: 6.1055 - accuracy: 0.5418 - val_loss: 4.3657 - val_accuracy: 0.5604
Epoch 5/5
5329/5329 [==============================] - 13s 2ms/step - loss: 6.1175 - accuracy: 0.5498 - val_loss: 4.7079 - val_accuracy: 0.5604


In [16]:
model = Sequential()
model.add(Conv1D(1024, kernel_size=1,
                 activation='relu',
                 input_shape=input_shape))
model.add(MaxPooling1D(pool_size=10))
model.add(Dropout(0.25))
model.add(Dense(512))
model.add(Conv1D(filters=32, kernel_size=1, activation='relu'))
model.add(MaxPooling1D(pool_size=1))
model.add(Conv1D(216, 1))
model.add(MaxPooling1D(pool_size=1))
model.add(Conv1D(128, 1))
model.add(MaxPooling1D(pool_size=1))
model.add(Conv1D(64, 1, activation='relu'))
model.add(MaxPooling1D(pool_size=1))
model.add(Conv1D(32, 1, activation='relu'))
model.add(MaxPooling1D(pool_size=1))
model.add(Flatten())
model.add(Dense(num_classes, activation='relu'))

model.compile(loss=keras.losses.binary_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

In [17]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_5 (Conv1D)            (None, 25, 1024)          2048      
_________________________________________________________________
max_pooling1d_4 (MaxPooling1 (None, 2, 1024)           0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 2, 1024)           0         
_________________________________________________________________
dense_4 (Dense)              (None, 2, 512)            524800    
_________________________________________________________________
conv1d_6 (Conv1D)            (None, 2, 32)             16416     
_________________________________________________________________
max_pooling1d_5 (MaxPooling1 (None, 2, 32)             0         
_________________________________________________________________
conv1d_7 (Conv1D)            (None, 2, 216)           

In [18]:
model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=5,
          verbose=1,
          validation_data=(X_test, y_test))

Epoch 1/5
5329/5329 [==============================] - 39s 7ms/step - loss: 2.4844 - accuracy: 0.5530 - val_loss: 0.6835 - val_accuracy: 0.4396
Epoch 2/5
5329/5329 [==============================] - 40s 7ms/step - loss: 1.4301 - accuracy: 0.5159 - val_loss: 0.6968 - val_accuracy: 0.4396
Epoch 3/5
5329/5329 [==============================] - 40s 7ms/step - loss: 1.1064 - accuracy: 0.5123 - val_loss: 0.6841 - val_accuracy: 0.4396
Epoch 4/5
5329/5329 [==============================] - 39s 7ms/step - loss: 0.9168 - accuracy: 0.4881 - val_loss: 0.6818 - val_accuracy: 0.4396
Epoch 5/5
5329/5329 [==============================] - 40s 7ms/step - loss: 0.8030 - accuracy: 0.4950 - val_loss: 0.6800 - val_accuracy: 0.4326


## Softmax

In [19]:
model = Sequential()
model.add(Conv1D(128, kernel_size=2,
                 activation='softmax',
                 input_shape=input_shape))
model.add(Conv1D(64, 1, activation='softmax'))
model.add(MaxPooling1D(pool_size=3))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='softmax'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.binary_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

In [20]:
model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=5,
          verbose=1,
          validation_data=(X_test, y_test))

Epoch 1/5
5329/5329 [==============================] - 18s 3ms/step - loss: 0.6932 - accuracy: 0.5179 - val_loss: 0.6931 - val_accuracy: 0.5604
Epoch 2/5
5329/5329 [==============================] - 17s 3ms/step - loss: 0.6932 - accuracy: 0.5059 - val_loss: 0.6931 - val_accuracy: 0.5604
Epoch 3/5
5329/5329 [==============================] - 18s 3ms/step - loss: 0.6932 - accuracy: 0.4982 - val_loss: 0.6931 - val_accuracy: 0.5604
Epoch 4/5
5329/5329 [==============================] - 19s 3ms/step - loss: 0.6932 - accuracy: 0.4849 - val_loss: 0.6931 - val_accuracy: 0.5604
Epoch 5/5
5329/5329 [==============================] - 17s 3ms/step - loss: 0.6932 - accuracy: 0.4932 - val_loss: 0.6931 - val_accuracy: 0.5604


## Tanh

In [21]:
model = Sequential()
model.add(Conv1D(128, kernel_size=2,
                 activation='tanh',
                 input_shape=input_shape))
model.add(Conv1D(64, 1, activation='tanh'))
model.add(MaxPooling1D(pool_size=3))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='tanh'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='tanh'))

model.compile(loss=keras.losses.binary_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

In [22]:
model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=5,
          verbose=1,
          validation_data=(X_test, y_test))

Epoch 1/5
5329/5329 [==============================] - 16s 3ms/step - loss: 1.7749 - accuracy: 0.4780 - val_loss: 0.7248 - val_accuracy: 0.4256
Epoch 2/5
5329/5329 [==============================] - 16s 3ms/step - loss: 2.0569 - accuracy: 0.4898 - val_loss: 0.6943 - val_accuracy: 0.4247
Epoch 3/5
5329/5329 [==============================] - 17s 3ms/step - loss: 2.1810 - accuracy: 0.4916 - val_loss: 0.6872 - val_accuracy: 0.4286
Epoch 4/5
5329/5329 [==============================] - 17s 3ms/step - loss: 2.2992 - accuracy: 0.4935 - val_loss: 0.6903 - val_accuracy: 0.4251
Epoch 5/5
5329/5329 [==============================] - 19s 3ms/step - loss: 2.3886 - accuracy: 0.4875 - val_loss: 0.6973 - val_accuracy: 0.4260


# Mix

In [50]:
model = Sequential()
model.add(Conv1D(128, kernel_size=1,
                 activation='relu',
                 input_shape=input_shape))
model.add(MaxPooling1D(pool_size=3))
model.add(Dropout(0.25))
model.add(Dense(128))
model.add(Conv1D(filters=32, kernel_size=1, activation='relu'))
model.add(MaxPooling1D(pool_size=3))
model.add(Conv1D(64, 1))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='relu'))

model.compile(loss=keras.losses.binary_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

In [51]:
model.summary()

Model: "sequential_18"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_54 (Conv1D)           (None, 25, 128)           256       
_________________________________________________________________
max_pooling1d_51 (MaxPooling (None, 8, 128)            0         
_________________________________________________________________
dropout_21 (Dropout)         (None, 8, 128)            0         
_________________________________________________________________
dense_33 (Dense)             (None, 8, 128)            16512     
_________________________________________________________________
conv1d_55 (Conv1D)           (None, 8, 32)             4128      
_________________________________________________________________
max_pooling1d_52 (MaxPooling (None, 2, 32)             0         
_________________________________________________________________
conv1d_56 (Conv1D)           (None, 2, 64)           

In [52]:
model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=5,
          verbose=1,
          validation_data=(X_test, y_test))

Epoch 1/5
5329/5329 [==============================] - 13s 2ms/step - loss: 7.0154 - accuracy: 0.5624 - val_loss: 5.9613 - val_accuracy: 0.5604
Epoch 2/5
5329/5329 [==============================] - 13s 2ms/step - loss: 6.9847 - accuracy: 0.5556 - val_loss: 5.1425 - val_accuracy: 0.5604
Epoch 3/5
5329/5329 [==============================] - 12s 2ms/step - loss: 7.0743 - accuracy: 0.5611 - val_loss: 3.7929 - val_accuracy: 0.5604
Epoch 4/5
5329/5329 [==============================] - 13s 2ms/step - loss: 7.0109 - accuracy: 0.5573 - val_loss: 3.7520 - val_accuracy: 0.5604
Epoch 5/5
5329/5329 [==============================] - 13s 2ms/step - loss: 6.9930 - accuracy: 0.5592 - val_loss: 3.7330 - val_accuracy: 0.5604


## Text w2v

In [3]:
train_set = pd.read_csv('train/text_encoded_w2v.csv')

In [4]:
X, y = train_set.iloc[:,:-1],train_set.iloc[:,-1]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

# Model

In [5]:
batch_size = 1
num_classes = 2
epochs = 10

In [6]:
X_train

,id,text_sum,text_min,text_max,text_mean,text_median
5535,7895,1.885115,-0.031004,0.262091,0.089767,0.059510
6018,8596,9.313427,-0.040400,1.000000,0.102345,0.069088
4739,6738,14.726642,0.012219,1.000000,0.188803,0.132279
3181,4567,3.985746,-0.040585,0.433890,0.110715,0.082433
818,1189,3.576633,-0.090845,0.443895,0.170316,0.168023
...,...,...,...,...,...,...
5282,7548,2.514225,-0.025919,1.000000,0.119725,0.074594
563,814,4.079575,-0.013456,1.000000,0.271972,0.073396
2089,3003,12.973744,-0.108944,1.000000,0.142569,0.114495
3221,4622,3.870319,-0.074206,0.450665,0.107509,0.085664


In [7]:
X_train = np.expand_dims(X_train, axis=2)
X_test = np.expand_dims(X_test, axis=2)

In [8]:
X_train.shape

(5329, 6, 1)

In [9]:
input_shape = (X_train.shape[1], 1)
input_shape

(6, 1)

## Relu

In [10]:
model = Sequential()
model.add(Conv1D(128, kernel_size=2,
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv1D(64, 1, activation='relu'))
model.add(MaxPooling1D(pool_size=3))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='relu'))

model.compile(loss=keras.losses.binary_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

In [11]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d (Conv1D)              (None, 5, 128)            384       
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 5, 64)             8256      
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 1, 64)             0         
_________________________________________________________________
dropout (Dropout)            (None, 1, 64)             0         
_________________________________________________________________
flatten (Flatten)            (None, 64)                0         
_________________________________________________________________
dense (Dense)                (None, 128)               8320      
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0

In [12]:
model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=5,
          verbose=1,
          validation_data=(X_test, y_test))

Epoch 1/5
5329/5329 [==============================] - 13s 2ms/step - loss: 8.2134 - accuracy: 0.4460 - val_loss: 8.4215 - val_accuracy: 0.4435
Epoch 2/5
5329/5329 [==============================] - 11s 2ms/step - loss: 8.2596 - accuracy: 0.4466 - val_loss: 8.4191 - val_accuracy: 0.4435
Epoch 3/5
5329/5329 [==============================] - 12s 2ms/step - loss: 8.1505 - accuracy: 0.4399 - val_loss: 8.4215 - val_accuracy: 0.4435
Epoch 4/5
5329/5329 [==============================] - 12s 2ms/step - loss: 8.2660 - accuracy: 0.4384 - val_loss: 8.4215 - val_accuracy: 0.4435
Epoch 5/5
5329/5329 [==============================] - 12s 2ms/step - loss: 8.2680 - accuracy: 0.4372 - val_loss: 8.4216 - val_accuracy: 0.4435


In [14]:
model = Sequential()
model.add(Conv1D(128, kernel_size=1,
                 activation='relu',
                 input_shape=input_shape))
model.add(MaxPooling1D(pool_size=1))
model.add(Dropout(0.25))
model.add(Dense(128))
model.add(Conv1D(filters=32, kernel_size=1, activation='relu'))
model.add(MaxPooling1D(pool_size=1))
model.add(Conv1D(64, 1))
model.add(MaxPooling1D(pool_size=1))
model.add(Flatten())
model.add(Dense(num_classes, activation='relu'))

model.compile(loss=keras.losses.binary_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

In [15]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_4 (Conv1D)            (None, 6, 128)            256       
_________________________________________________________________
max_pooling1d_3 (MaxPooling1 (None, 6, 128)            0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 6, 128)            0         
_________________________________________________________________
dense_3 (Dense)              (None, 6, 128)            16512     
_________________________________________________________________
conv1d_5 (Conv1D)            (None, 6, 32)             4128      
_________________________________________________________________
max_pooling1d_4 (MaxPooling1 (None, 6, 32)             0         
_________________________________________________________________
conv1d_6 (Conv1D)            (None, 6, 64)            

In [16]:
model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=5,
          verbose=1,
          validation_data=(X_test, y_test))

Epoch 1/5
5329/5329 [==============================] - 14s 3ms/step - loss: 7.9467 - accuracy: 0.5384 - val_loss: 8.3027 - val_accuracy: 0.5565
Epoch 2/5
5329/5329 [==============================] - 13s 2ms/step - loss: 7.9275 - accuracy: 0.5485 - val_loss: 8.2378 - val_accuracy: 0.5565
Epoch 3/5
5329/5329 [==============================] - 14s 3ms/step - loss: 7.8626 - accuracy: 0.5372 - val_loss: 8.1313 - val_accuracy: 0.5565
Epoch 4/5
5329/5329 [==============================] - 13s 2ms/step - loss: 7.8740 - accuracy: 0.5393 - val_loss: 7.7117 - val_accuracy: 0.5565
Epoch 5/5
5329/5329 [==============================] - 13s 2ms/step - loss: 7.8631 - accuracy: 0.5378 - val_loss: 7.4220 - val_accuracy: 0.5565


In [17]:
model = Sequential()
model.add(Conv1D(1024, kernel_size=1,
                 activation='relu',
                 input_shape=input_shape))
model.add(MaxPooling1D(pool_size=1))
model.add(Dropout(0.25))
model.add(Dense(512))
model.add(Conv1D(filters=32, kernel_size=1, activation='relu'))
model.add(MaxPooling1D(pool_size=1))
model.add(Conv1D(216, 1))
model.add(MaxPooling1D(pool_size=1))
model.add(Conv1D(128, 1))
model.add(MaxPooling1D(pool_size=1))
model.add(Conv1D(64, 1, activation='relu'))
model.add(MaxPooling1D(pool_size=1))
model.add(Conv1D(32, 1, activation='relu'))
model.add(MaxPooling1D(pool_size=1))
model.add(Flatten())
model.add(Dense(num_classes, activation='relu'))

model.compile(loss=keras.losses.binary_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

In [18]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_7 (Conv1D)            (None, 6, 1024)           2048      
_________________________________________________________________
max_pooling1d_6 (MaxPooling1 (None, 6, 1024)           0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 6, 1024)           0         
_________________________________________________________________
dense_5 (Dense)              (None, 6, 512)            524800    
_________________________________________________________________
conv1d_8 (Conv1D)            (None, 6, 32)             16416     
_________________________________________________________________
max_pooling1d_7 (MaxPooling1 (None, 6, 32)             0         
_________________________________________________________________
conv1d_9 (Conv1D)            (None, 6, 216)           

In [19]:
model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=5,
          verbose=1,
          validation_data=(X_test, y_test))

Epoch 1/5
5329/5329 [==============================] - 47s 9ms/step - loss: 8.0898 - accuracy: 0.4265 - val_loss: 7.5757 - val_accuracy: 0.4426
Epoch 2/5
5329/5329 [==============================] - 51s 10ms/step - loss: 7.9531 - accuracy: 0.4260 - val_loss: 7.6082 - val_accuracy: 0.4426
Epoch 3/5
5329/5329 [==============================] - 50s 9ms/step - loss: 7.8164 - accuracy: 0.4218 - val_loss: 7.6192 - val_accuracy: 0.4426
Epoch 4/5
5329/5329 [==============================] - 47s 9ms/step - loss: 7.7759 - accuracy: 0.4256 - val_loss: 7.6203 - val_accuracy: 0.4431
Epoch 5/5
5329/5329 [==============================] - 48s 9ms/step - loss: 7.6847 - accuracy: 0.4237 - val_loss: 7.6232 - val_accuracy: 0.4431


## Softmax

In [20]:
model = Sequential()
model.add(Conv1D(128, kernel_size=2,
                 activation='softmax',
                 input_shape=input_shape))
model.add(Conv1D(64, 1, activation='softmax'))
model.add(MaxPooling1D(pool_size=3))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='softmax'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.binary_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

In [21]:
model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=5,
          verbose=1,
          validation_data=(X_test, y_test))

Epoch 1/5
5329/5329 [==============================] - 13s 2ms/step - loss: 0.6932 - accuracy: 0.4915 - val_loss: 0.6931 - val_accuracy: 0.4435
Epoch 2/5
5329/5329 [==============================] - 12s 2ms/step - loss: 0.6932 - accuracy: 0.4992 - val_loss: 0.6931 - val_accuracy: 0.4435
Epoch 3/5
5329/5329 [==============================] - 11s 2ms/step - loss: 0.6932 - accuracy: 0.5050 - val_loss: 0.6931 - val_accuracy: 0.4435
Epoch 4/5
5329/5329 [==============================] - 12s 2ms/step - loss: 0.6932 - accuracy: 0.4864 - val_loss: 0.6931 - val_accuracy: 0.4435
Epoch 5/5
5329/5329 [==============================] - 12s 2ms/step - loss: 0.6932 - accuracy: 0.5042 - val_loss: 0.6931 - val_accuracy: 0.4435


## Tanh

In [22]:
model = Sequential()
model.add(Conv1D(128, kernel_size=2,
                 activation='tanh',
                 input_shape=input_shape))
model.add(Conv1D(64, 1, activation='tanh'))
model.add(MaxPooling1D(pool_size=3))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='tanh'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='tanh'))

model.compile(loss=keras.losses.binary_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

In [23]:
model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=5,
          verbose=1,
          validation_data=(X_test, y_test))

Epoch 1/5
5329/5329 [==============================] - 11s 2ms/step - loss: 3.2916 - accuracy: 0.4843 - val_loss: 0.8663 - val_accuracy: 0.4426
Epoch 2/5
5329/5329 [==============================] - 11s 2ms/step - loss: 3.1247 - accuracy: 0.4881 - val_loss: 0.8411 - val_accuracy: 0.4435
Epoch 3/5
5329/5329 [==============================] - 12s 2ms/step - loss: 3.1101 - accuracy: 0.4935 - val_loss: 0.8166 - val_accuracy: 0.4448
Epoch 4/5
5329/5329 [==============================] - 12s 2ms/step - loss: 3.0732 - accuracy: 0.4817 - val_loss: 0.8006 - val_accuracy: 0.4453
Epoch 5/5
5329/5329 [==============================] - 12s 2ms/step - loss: 2.9780 - accuracy: 0.5044 - val_loss: 0.7842 - val_accuracy: 0.4448
